In [37]:
import numpy as np
from scipy.spatial import distance_matrix, minkowski_distance_p

In [19]:
def psquare(x):
    mat = squareform(pdist(x))
    return mat


def dismat(x):
    mat = distance_matrix(x, x)
    return mat


def mindist(x):
    mat = minkowski_distance_p(x[:, np.newaxis, :], x[np.newaxis, :, :], 2) ** (1 / 2)
    return mat


def manual(x):
    mat = np.matmul(x, x.T)
    x2 = np.sum(np.square(x), axis=1)

    for i in range(mat.shape[0]):
        mat[i, :] = np.multiply(mat[i, :], -2)
        mat[i, :] = np.add(mat[i, :], x2.T[i])
        mat[i, :] = np.add(mat[i, :], x2)

    np.fill_diagonal(mat, 0)
    return np.sqrt(mat)


def opt_man(x):
    # Compute the squared magnitudes of each vector
    x2 = np.sum(np.square(x), axis=1, keepdims=True)
    # Compute the matrix product between x and its transpose
    mat = np.matmul(x, x.T)
    # Utilize broadcasting to perform the subtraction and addition operations
    mat *= -2
    mat += x2
    mat += x2.T
    # Ensure the diagonal elements are 0 and not a tiny negative number due to floating point arithmetic issues
    np.fill_diagonal(mat, 0)
    # Square root of the matrix to get the Euclidean distances
    return np.sqrt(mat)

In [4]:
rng = np.random.default_rng(33)

In [8]:
x10_256 = rng.random((10000, 256))
x10_512 = rng.random((10000, 512))
x10_1024 = rng.random((10000, 1024))

x30_256 = rng.random((30000, 256))
x30_512 = rng.random((30000, 512))
x30_1024 = rng.random((30000, 1024))

In [4]:
%timeit psquare(x10_256)

5.36 s ± 97.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [5]:
%timeit psquare(x10_512)

10.7 s ± 132 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [6]:
%timeit psquare(x10_1024)

20.8 s ± 167 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [7]:
%timeit dismat(x10_256)

1min 56s ± 783 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [9]:
%timeit manual(x10_256)

890 ms ± 6.19 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [10]:
%timeit manual(x10_512)

947 ms ± 11.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [20]:
%timeit psquare(x30_256)

1min 22s ± 669 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [22]:
%timeit opt_man(x10_1024)

1.5 s ± 17.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [11]:
%timeit manual(x10_1024)

1.15 s ± 25.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [19]:
%timeit manual(x30_1024)

10 s ± 114 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [33]:
%timeit opt_man(x30_1024)

11.2 s ± 210 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [34]:
%timeit opt_man(x10_1024)

1.2 s ± 23 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [35]:
checko = opt_man(x10_512)
assert checko.all() == checkp.all()

In [5]:
x50_1024 = rng.random((50000, 1024), dtype=np.float32)

In [6]:
%timeit opt_man(x50_1024)

16 s ± 213 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [7]:
%timeit manual(x50_1024)

14.8 s ± 201 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [14]:
%timeit manual(x30_1024)

11.4 s ± 240 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [19]:
%timeit opt_man(x30_1024)

13.6 s ± 175 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [20]:
checkp = psquare(x10_512)
checko = opt_man(x10_512)
checkm = manual(x10_512)

print(checkp[:5, :5])
print(checko[:5, :5])
print(checkm[:5, :5])

[[0.         9.05017016 8.90532485 9.19391913 9.57348328]
 [9.05017016 0.         9.89339359 9.64133312 9.07223626]
 [8.90532485 9.89339359 0.         9.23835396 9.33599252]
 [9.19391913 9.64133312 9.23835396 0.         8.91816132]
 [9.57348328 9.07223626 9.33599252 8.91816132 0.        ]]
[[0.         9.05017016 8.90532485 9.19391913 9.57348328]
 [9.05017016 0.         9.89339359 9.64133312 9.07223626]
 [8.90532485 9.89339359 0.         9.23835396 9.33599252]
 [9.19391913 9.64133312 9.23835396 0.         8.91816132]
 [9.57348328 9.07223626 9.33599252 8.91816132 0.        ]]
[[0.         9.05017016 8.90532485 9.19391913 9.57348328]
 [9.05017016 0.         9.89339359 9.64133312 9.07223626]
 [8.90532485 9.89339359 0.         9.23835396 9.33599252]
 [9.19391913 9.64133312 9.23835396 0.         8.91816132]
 [9.57348328 9.07223626 9.33599252 8.91816132 0.        ]]


In [21]:
from sklearn.metrics import pairwise_distances

distance_matrix = pairwise_distances(x10_512, metric="euclidean")
print(distance_matrix[:5, :5])

[[0.         9.05017016 8.90532485 9.19391913 9.57348328]
 [9.05017016 0.         9.89339359 9.64133312 9.07223626]
 [8.90532485 9.89339359 0.         9.23835396 9.33599252]
 [9.19391913 9.64133312 9.23835396 0.         8.91816132]
 [9.57348328 9.07223626 9.33599252 8.91816132 0.        ]]


In [22]:
%timeit pairwise_distances(x30_1024)

12 s ± 174 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [23]:
import faiss

x = x10_1024.astype(np.float32)
index = faiss.IndexFlatL2(x.shape[1])  # L2 distance
index.add(x)
D, I = index.search(x, k=x.shape[0])
print(D.shape)
print(I.shape)

ModuleNotFoundError: No module named 'faiss'

Faiss on the cpu is not faster if everything fits into memory. Faiss comes into play with larger datasets and the use of the gpu.


Could adjust mst to use linkage and just create labels for new rows such that labels are aggregated for clusters, allows easy combining with cleaning code and cluster formation/mislabeling.


# Testing Starts Here


In [1]:
import hashlib
import os
import typing
from copy import deepcopy
from urllib.error import HTTPError, URLError
from urllib.request import urlretrieve

import numpy as np
from scipy.cluster.hierarchy import linkage
from scipy.spatial.distance import pdist, squareform

In [2]:
def _validate_file(fpath, file_hash, chunk_size=65535):
    hasher = hashlib.sha256()
    with open(fpath, "rb") as fpath_file:
        for chunk in iter(lambda: fpath_file.read(chunk_size), b""):
            hasher.update(chunk)

    return str(hasher.hexdigest()) == str(file_hash)


def _get_file(
    fname: str,
    origin: str,
    file_hash: typing.Optional[str] = None,
):
    cache_dir = os.path.join(os.path.expanduser("~"), ".keras")
    datadir_base = os.path.expanduser(cache_dir)
    if not os.access(datadir_base, os.W_OK):
        datadir_base = os.path.join("/tmp", ".keras")
    datadir = os.path.join(datadir_base, "datasets")
    os.makedirs(datadir, exist_ok=True)

    fname = os.fspath(fname) if isinstance(fname, os.PathLike) else fname
    fpath = os.path.join(datadir, fname)

    download = False
    if os.path.exists(fpath):
        if file_hash is not None and not _validate_file(fpath, file_hash):
            download = True
    else:
        download = True

    if download:
        try:
            error_msg = "URL fetch failure on {}: {} -- {}"
            try:
                urlretrieve(origin, fpath)
            except HTTPError as e:
                raise Exception(error_msg.format(origin, e.code, e.msg)) from e
            except URLError as e:
                raise Exception(error_msg.format(origin, e.errno, e.reason)) from e
        except (Exception, KeyboardInterrupt):
            if os.path.exists(fpath):
                os.remove(fpath)
            raise

        if (
            os.path.exists(fpath)
            and file_hash is not None
            and not _validate_file(fpath, file_hash)
        ):
            raise ValueError(
                "Incomplete or corrupted file detected. "
                f"The sha256 file hash does not match the provided value "
                f"of {file_hash}.",
            )
    return fpath

In [3]:
origin_folder = "https://storage.googleapis.com/tensorflow/tf-keras-datasets/"
path = _get_file(
    "mnist.npz",
    origin=origin_folder + "mnist.npz",
    file_hash=("731c5ac602752760c8e48fbffcf8c3b850d9dc2a2aedcf2cc48468fc17b673d1"),
)

with np.load(path, allow_pickle=True) as fp:
    images, labels = fp["x_train"][:100], fp["y_train"][:100]

In [4]:
rng = np.random.default_rng(33)

In [5]:
images.shape

(100, 28, 28)

In [6]:
dup_images = deepcopy(images[:8])
dup_images[:, :21, :21] = images[:8, 7:, 7:]
dup_images[:, 21:, 21:] = images[:8, :7, :7]

In [7]:
test_img = np.concatenate([images, dup_images])

In [8]:
test_img.shape

(108, 28, 28)

In [9]:
rng.shuffle(test_img)

In [10]:
def square_to_condensed(square_matrix):
    """Convert a square distance matrix to a condensed distance matrix."""
    assert square_matrix.shape[0] == square_matrix.shape[1], "Matrix must be square"
    n = square_matrix.shape[0]
    tri_indices = np.triu_indices(n, 1)
    condensed_matrix = square_matrix[tri_indices]
    return condensed_matrix


def L2_distance_matrix_sq(x):
    """
    Takes advantage that Euclidean distance can be written as
    ||a-b||**2 = ||a||**2 + ||b||**2 - 2 a @ b
    Also, does not compute the sqrt at the end since the squared distance
    is still going to produce an equivalent nearest neighbor matrix
    """
    # Adjusting dtype to be friendlier for speed and memory
    # x = x.astype(np.float32)
    # Compute the squared magnitude of the vector
    x2 = np.sum(np.square(x), axis=1, keepdims=True, dtype=x.dtype)
    # Compute the matrix product between x and its transpose
    mat = np.matmul(x, x.T) * -2
    # Utilize broadcasting to perform the addition
    mat += x2
    mat += x2.T

    return square_to_condensed(np.abs(mat))


def sort_linkage(Z):
    """
    Sort the linkage matrix Z in reverse order by distance and
    then by cluster size (new_size).

    Parameters:
    - arr: linkage matrix

    Returns:
    - arr: Sorted linkage matrix
    """
    # Adjusting linkage matrix to accomodate renumbering
    arr = np.zeros((Z.shape[0], Z.shape[1] + 1))
    arr[:, :-1] = Z.copy()
    arr[:, -1] = np.arange(Z.shape[0] + 1, 2 * Z.shape[0] + 1)

    # Sort by decreasing distance, then by increasing new_size
    # arr = arr[arr[:, 2].argsort()[::-1]]
    # arr = arr[arr[:, -2].argsort(kind="stable")]

    return arr

In [30]:
dist_mat = pdist(test_img.reshape((test_img.shape[0], -1)), metric="euclidean")
Z = linkage(dist_mat)
Zsort = sort_linkage(Z)

In [32]:
def get_duplicate(link_arr, distance):
    link_std = link_arr.std()

    if distance <= link_std / 1e3:
        return "exact duplicate"
    elif distance <= link_std:
        return "near duplicate"
    else:
        return ""


def get_outlier(level, distance, dist_arr):
    dist_mean, dist_std = dist_arr[level].mean(), dist_arr[level].std()

    if abs(dist_mean - distance) < dist_std * 2:
        return "outlier"
    elif level >= dist_arr.shape[0] * 2 / 3:
        return "potential outlier"
    else:
        return ""


def get_distance(
    cluster, level, sample, distance_array, distance_matrix, sample_clusters
):
    # Convert the condensed distance matrix to a square form
    square_distance_matrix = squareform(distance_matrix)

    # For each cluster, check if it is active at the current level
    for cluster_num, level_info in sample_clusters.items():
        if cluster_num != cluster and level in level_info:
            samples = level_info[level]["samples"]

            for other_sample in samples:
                if (
                    square_distance_matrix[sample, other_sample]
                    < distance_array[cluster]
                ):
                    distance_array[cluster] = square_distance_matrix[
                        sample, other_sample
                    ]

    return distance_array


def reorganize_clusters(clusters):
    """
    Reorganize the clusters dictionary to be nested by cluster_num, then by level,
    and include avg_dist, sample_dist, and samples within each level.

    Parameters:
    - clusters: A dictionary containing the original clusters information.

    Returns:
    - new_structure: A dictionary reorganized by cluster_num,
                      then by level, with details.
    """
    new_structure = {}

    for _, info in clusters.items():
        # Extract necessary information
        cluster_num = info["cluster_num"]
        level = info["level"]
        samples = info.get("samples_added", [])

        # Initialize the structure if not present
        if cluster_num not in new_structure:
            new_structure[cluster_num] = {}

        if level not in new_structure[cluster_num] and level == 1:
            new_structure[cluster_num][level] = {"samples": []}
        elif level not in new_structure[cluster_num] and level > 1:
            new_structure[cluster_num][level] = {
                "samples": deepcopy(new_structure[cluster_num][level - 1]["samples"])
            }

        # Extending the samples list.
        new_structure[cluster_num][level]["samples"].extend(samples)

    return new_structure


def get_sample_info(arr, distance_matrix):
    """
    Initialize clusters based on number of individual sample merges.

    Parameters:
    - arr: sorted linkage matrix

    Returns:
    - clusters: A dictionary containing the clusters
    """
    # Determining maximum number of levels and clusters
    max_clusters = 1
    max_levels = 1
    clusters = {}
    for i in range(len(arr)):
        level = 1
        cluster_num = max_clusters
        distance = 0
        count = 0
        sample_added = []
        if arr[i, 0] in clusters:
            cluster_num = min([cluster_num, clusters[arr[i, 0]]["cluster_num"]])
            left_level = max([level, clusters[arr[i, 0]]["level"] + 1])
            distance += clusters[arr[i, 0]]["total_dist"]
            count += clusters[arr[i, 0]]["count"]
        else:
            sample_added.append(int(arr[i, 0]))

        if arr[i, 1] in clusters:
            cluster_num = min([cluster_num, clusters[arr[i, 1]]["cluster_num"]])
            right_level = max([level, clusters[arr[i, 1]]["level"] + 1])
            distance += clusters[arr[i, 1]]["total_dist"]
            count += clusters[arr[i, 1]]["count"]
        else:
            sample_added.append(int(arr[i, 1]))

        if arr[i, 0] in clusters and arr[i, 1] in clusters:
            if cluster_num == clusters[arr[i, 0]]["cluster_num"]:
                level = left_level
            elif cluster_num == clusters[arr[i, 1]]["cluster_num"]:
                level = right_level
        elif arr[i, 0] in clusters:
            level = left_level
        elif arr[i, 1] in clusters:
            level = right_level

        count += 1
        distance += arr[i, 2]

        clusters[arr[i, -1]] = {
            "cluster_num": cluster_num,
            "level": level,
            "total_dist": distance,
            "count": count,
            "avg_dist": distance / count,
            "samples_added": sample_added,
            "sample_dist": arr[i, 2],
        }

        if cluster_num == max_clusters and i < len(arr) - 1:
            max_clusters += 1

        if level > max_levels:
            max_levels = level

    # Reorganizing the clusters dictionary
    sample_clusters = reorganize_clusters(clusters)

    # Creating the cluster tracking dictionary
    sample_tracking = {
        i: {
            "cluster": np.zeros(max_levels),
            "distance": np.full((max_levels, max_clusters), np.inf),
            "duplicate": "",
            "outlier": "",
        }
        for i in range(len(arr) + 1)
    }

    for _, values in clusters.items():
        if values["samples_added"]:
            level = values["level"] - 1
            cluster = values["cluster_num"] - 1
            for sample in values["samples_added"]:
                sample_tracking[sample]["cluster"][level] = values["cluster_num"]
                sample_tracking[sample]["distance"][level, cluster] = values[
                    "sample_dist"
                ]
                sample_tracking[sample]["distance"][level, :] = get_distance(
                    cluster,
                    level,
                    sample,
                    sample_tracking[sample]["distance"][level, :],
                    distance_matrix,
                    sample_clusters,
                )
                sample_tracking[sample]["duplicate"] = get_duplicate(
                    Zsort[:, 2],
                    values["sample_dist"],
                )
                sample_tracking[sample]["outlier"] = get_outlier(
                    level,
                    values["sample_dist"],
                    sample_tracking[sample]["distance"],
                )

    return clusters, sample_tracking

In [33]:
clusters, sample_tracking = get_sample_info(Zsort, dist_mat)

In [34]:
for _, values in clusters.items():
    if values["samples_added"]:
        print(values)
        break

{'cluster_num': 1, 'level': 1, 'total_dist': 419.583126448145, 'count': 1, 'avg_dist': 419.583126448145, 'samples_added': [13, 107], 'sample_dist': 419.583126448145}


In [35]:
Zsort

array([[1.30000000e+01, 1.07000000e+02, 4.19583126e+02, 2.00000000e+00,
        1.08000000e+02],
       [8.20000000e+01, 1.08000000e+02, 8.09468344e+02, 3.00000000e+00,
        1.09000000e+02],
       [2.30000000e+01, 4.50000000e+01, 9.43248642e+02, 2.00000000e+00,
        1.10000000e+02],
       [4.20000000e+01, 1.10000000e+02, 9.64527864e+02, 3.00000000e+00,
        1.11000000e+02],
       [3.70000000e+01, 1.09000000e+02, 9.68168890e+02, 4.00000000e+00,
        1.12000000e+02],
       [3.00000000e+01, 1.11000000e+02, 9.87199575e+02, 4.00000000e+00,
        1.13000000e+02],
       [6.50000000e+01, 7.10000000e+01, 1.03831017e+03, 2.00000000e+00,
        1.14000000e+02],
       [3.20000000e+01, 9.70000000e+01, 1.04582121e+03, 2.00000000e+00,
        1.15000000e+02],
       [1.90000000e+01, 6.10000000e+01, 1.10696883e+03, 2.00000000e+00,
        1.16000000e+02],
       [3.60000000e+01, 8.60000000e+01, 1.12729056e+03, 2.00000000e+00,
        1.17000000e+02],
       [4.10000000e+01, 1.1200

In [37]:
sample_tracking[13]

{'cluster': array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0.]),
 'distance': array([[419.58312645,          inf,          inf, ...,          inf,
                  inf,          inf],
        [         inf,          inf,          inf, ...,          inf,
                  inf,          inf],
        [         inf,          inf,          inf, ...,          inf,
                  inf,          inf],
        ...,
        [         inf,          inf,          inf, ...,          inf,
                  inf,          inf],
        [         inf,          inf,          inf, ...,          inf,
                  inf,          inf],
        [         inf,          inf,          inf, ...,          inf,
                  inf,          inf]]),
 'duplicate': '',
 'outlier': ''}

In [106]:
len(sample_tracking[205]["cluster"])

1071

In [100]:
sample_tracking[205]

{'cluster': array([ 0., 13.,  0., ...,  0.,  0.,  0.]),
 'distance': array([[inf, inf, inf, ..., inf, inf, inf],
        [inf, inf, inf, ..., inf, inf, inf],
        [inf, inf, inf, ..., inf, inf, inf],
        ...,
        [inf, inf, inf, ..., inf, inf, inf],
        [inf, inf, inf, ..., inf, inf, inf],
        [inf, inf, inf, ..., inf, inf, inf]]),
 'duplicate': '',
 'outlier': ''}